<a href="https://colab.research.google.com/github/meduardajustino/Estudos-Python/blob/main/05_transformacao_de_dados_2025_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformação de dados
---

In [ ]:
import pandas as pd

# pipelines e transformadores
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# codificação de variáveis
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.feature_extraction.text import CountVectorizer

# normalização
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

# imputação
from sklearn.impute import SimpleImputer

# modelo
from sklearn.linear_model import LinearRegression

# para facilitar os exemplos
from sklearn import set_config
set_config(transform_output = "pandas")

In [ ]:
# @title Leitura do conjunto de dados

df = pd.read_csv("https://raw.githubusercontent.com/atlantico-academy/datasets/refs/heads/main/tips.csv")

In [ ]:
# @title Dicionário de dados
df_dict = pd.DataFrame([
    {
        "variavel": "total_bill",
        "descricao": "Total pago da conta em dólares.",
        "tipo": "quantitativa",
        "subtipo": "contínua",
    },
    {
        "variavel": "tip",
        "descricao": "Valor da gorjeta dada ao garçom em dólares.",
        "tipo": "quantitativa",
        "subtipo": "contínua",
    },
    {
        "variavel": "sex",
        "descricao": "Gênero do cliente (Male/Female).",
        "tipo": "qualitativa",
        "subtipo": "nominal",
    },
    {
        "variavel": "smoker",
        "descricao": "Indica se o cliente é fumante (Yes/No).",
        "tipo": "qualitativa",
        "subtipo": "nominal",
    },
    {
        "variavel": "day",
        "descricao": "Dia da semana da refeição (Thur, Fri, Sat, Sun).",
        "tipo": "qualitativa",
        "subtipo": "ordinal",
    },
    {
        "variavel": "time",
        "descricao": "Período do dia em que ocorreu a refeição (Lunch/Dinner).",
        "tipo": "qualitativa",
        "subtipo": "ordinal",
    },
    {
        "variavel": "size",
        "descricao": "Número de pessoas na mesa.",
        "tipo": "quantitativa",
        "subtipo": "discreta",
    }
])
df_dict

,variavel,descricao,tipo,subtipo
0,total_bill,Total pago da conta em dólares.,quantitativa,contínua
1,tip,Valor da gorjeta dada ao garçom em dólares.,quantitativa,contínua
2,sex,Gênero do cliente (Male/Female).,qualitativa,nominal
3,smoker,Indica se o cliente é fumante (Yes/No).,qualitativa,nominal
4,day,"Dia da semana da refeição (Thur, Fri, Sat, Sun).",qualitativa,ordinal
5,time,Período do dia em que ocorreu a refeição (Lunc...,qualitativa,ordinal
6,size,Número de pessoas na mesa.,quantitativa,discreta


In [ ]:
# definir variável alvo
target_variable = "tip"
# selecionar variáveis não utilizadas
unused_variables = [target_variable]

# selecionar variáveis qualitativas nominais
nominal_variables = (
    df_dict
    .query("tipo == 'qualitativa' and subtipo == 'nominal' and variavel != @target_variable")
    .variavel
    .to_list()
)
# selecionar variáveis qualitativas ordinais
ordinal_variables = (
    df_dict
    .query("tipo == 'qualitativa' and subtipo == 'ordinal' and variavel != @target_variable")
    .variavel
    .to_list()
)
# selecionar variáveis quantitativas discretas
discrete_variables = (
    df_dict
    .query("tipo == 'quantitativa' and subtipo == 'discreta' and variavel != @target_variable")
    .variavel
    .to_list()
)
# selecionar variáveis quantitativas contínuas
continuous_variables = (
    df_dict
    .query("tipo == 'quantitativa' and subtipo == 'contínua' and variavel != @target_variable")
    .variavel
    .to_list()
)

# Criar X e y
X = df.drop(columns=unused_variables)
y = df[target_variable]

In [ ]:
# @title Preprocessameto

# variáveis ordinal
ordinal_preprocessing = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')),
    ("encoding", OrdinalEncoder(categories=[['Thur', 'Fri', 'Sat', 'Sun'], ['Lunch', 'Dinner']])),
    # ("normalization", StandardScaler())
])

# variáveis nominais
nominal_preprocessing = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')),
    ("encoding", OneHotEncoder(sparse_output=False)),
    # ("normalization", StandardScaler())
])

# variáveis discretas
discrete_preprocessing = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='median')),
    # ("normalization", StandardScaler())
])

# variáveis contínuas
continuous_preprocessing = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='mean')),
    # ("normalization", StandardScaler())
])

In [ ]:
preprocessing = ColumnTransformer(transformers=[
    ("ordinal", ordinal_preprocessing, ordinal_variables),
    ("nominal", nominal_preprocessing, nominal_variables),
    ("discrete", discrete_preprocessing, discrete_variables),
    ("continuous", continuous_preprocessing, continuous_variables),
])

In [ ]:
approach = Pipeline(steps=[
    ("preprocessing", preprocessing),
    ("normalizer", StandardScaler()),
    ("model", LinearRegression()),
])

In [ ]:
approach.fit(X, y)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('ordinal',
                                                  Pipeline(steps=[('missing',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoding',
                                                                   OrdinalEncoder(categories=[['Thur',
                                                                                               'Fri',
                                                                                               'Sat',
                                                                                               'Sun'],
                                                                                              ['Lunch',
                                                                                               'Dinner']]))]),
                                                  ['day', 'time']),
                                                 ('nominal',
                                                  Pipeline(steps=[('missing',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoding',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['sex', 'smoker']),
                                                 ('discrete',
                                                  Pipeline(steps=[('missing',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['size']),
                                                 ('continuous',
                                                  Pipeline(steps=[('missing',
                                                                   SimpleImputer())]),
                                                  ['total_bill'])])),
                ('normalizer', StandardScaler()),
                ('model', LinearRegression())])

In [ ]:
approach.predict(X)

array([2.72196753, 2.23488035, 3.24132904, 3.3183574 , 3.78847743,
       3.81986003, 1.91197035, 3.96983692, 2.50338865, 2.47886412,
       2.05345798, 4.79492612, 2.53923218, 3.17278993, 2.51822534,
       3.12027472, 2.26858206, 2.79611462, 2.89490065, 3.15463219,
       2.72230508, 2.9805005 , 2.5541511 , 5.0999337 , 2.90152274,
       3.06156855, 2.29312593, 2.22898487, 3.07885391, 2.92013244,
       1.93280409, 3.1125041 , 2.48718029, 3.36786977, 2.70909957,
       3.47628073, 2.7452613 , 2.83838782, 2.96975501, 4.15636462,
       2.71979353, 2.73165536, 2.39963105, 1.99780618, 4.30186123,
       2.80994518, 3.18158603, 4.4905114 , 3.9525402 , 2.78636391,
       2.26757593, 2.08998944, 4.75247983, 2.0223307 , 3.8453278 ,
       2.92313529, 4.89127224, 3.55777004, 2.0165504 , 5.93471073,
       2.87019245, 2.25896588, 1.99579888, 3.03118151, 2.86599742,
       3.10086689, 2.61829216, 1.10573978, 2.94019603, 2.37215598,
       2.16578706, 2.85159334, 3.52455324, 3.3755196 , 2.45605